In [1]:
# !git clone https://github.com/guilevieiram/title-generation.git
# !pip install datasets
# !pip install git+https://github.com/guilevieiram/title-generation.git
# !pip install transformers[torch]
# !pip install evaluate
# !pip install trl
# !pip install peft

In [2]:
from datasets import load_dataset

import pandas as pd
import numpy as np

import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import evaluate

from peft import LoraConfig, get_peft_model

import nltk
from nltk.tokenize import sent_tokenize

nltk.download("punkt")
nltk.download('stopwords')

2024-03-16 22:21:38.578675: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-16 22:21:38.578699: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-16 22:21:38.579323: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-16 22:21:38.583110: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-16 22:21:39.633103: W tensorflow/compiler/tf2

True

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
rouge_score = evaluate.load("rouge")

#data
crop=0.1

#base model
def base_prompt(text):
    return f"Créez un titre: \n{text}"
base_model_checkpoint = "/Data/mt0-large-r128-e10-c0.3-finetune"
temperature = 2.0
base_max_input_length = 896
base_batch_size = 4
ensamble_size=10

#model training
model_checkpoint = "bigscience/mt0-large"
r=8
num_train_epochs = 50
batch_size = 2
max_input_length = 512
max_target_length = 64
finetune_att=True
finetune_lin=True
model_name = f"{base_model_checkpoint.split('/')[1]}-r{r}-e{num_train_epochs}-c{crop}-ensamble"
out_dir = f"/Data/{model_name}"



In [4]:
dataset = load_dataset('csv', data_files={'train': '../data/train.csv', 'validation': '../data/validation.csv'})
train_sample_size = int(crop * len(dataset['train']))
validation_sample_size = int(crop * len(dataset['validation']))
dataset['train'] = dataset['train'].shuffle().select(range(train_sample_size))
dataset['validation'] = dataset['validation'].shuffle().select(range(validation_sample_size))


In [5]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_checkpoint)
base_model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): lora.Linear(
                (base_layer): Linear(in_

In [7]:
from transformers import BatchEncoding
def preprocess_function(examples):
    batch_s=len(examples['text'])
    base_model_inputs = tokenizer(
        [base_prompt(text) for text in examples["text"]],
        max_length=base_max_input_length,
        truncation=True,
        padding=True,
        return_tensors='pt'
    ).to(device)
    output_tokens = base_model.generate(
        **base_model_inputs, 
        max_new_tokens=max_target_length, 
        temperature=temperature,
        do_sample=True,
        num_return_sequences=ensamble_size
    ).view(batch_s, -1)
    model_inputs=BatchEncoding({'input_ids':output_tokens})
    labels = tokenizer(
        examples["titles"], max_length=max_target_length, truncation=True, padding='max_length', return_tensors='pt'
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True, batch_size=base_batch_size)

Map:   0%|          | 0/2140 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

In [8]:
base_model.to('cpu')

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 1024)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 1024)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=128, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=128, out_features=1024, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k): lora.Linear(
                (base_layer): Linear(in_

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
print_trainable_parameters(model)
model.to(device)

att_target_modules = ["q", "k", "v", "o"]
import re
pattern = r'\((\w+)\): Linear'
linear_layers = re.findall(pattern, str(model.modules))
lin_target_modules = list(set(linear_layers))

target_modules = []
if finetune_att: target_modules.extend(att_target_modules)
if finetune_lin: target_modules.extend(lin_target_modules)

lora_config = LoraConfig(
    r=r,
    target_modules = target_modules,
    lora_alpha=8,
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

model

trainable params: 1229581312 || all params: 1229581312 || trainable%: 100.0
trainable params: 11151360 || all params: 1240732672 || trainable%: 0.8987721732212111


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 1024)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 1024)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
 

In [10]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    result = {k: round(v, 4) for k, v in result.items()}
    print(result)
    return result

args = Seq2SeqTrainingArguments(
    output_dir=out_dir,
    evaluation_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
)
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model),
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [11]:
trainer.evaluate()

/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.1617, 'rouge2': 0.0395, 'rougeL': 0.1256, 'rougeLsum': 0.132}


{'eval_loss': 6.949134349822998,
 'eval_rouge1': 0.1617,
 'eval_rouge2': 0.0395,
 'eval_rougeL': 0.1256,
 'eval_rougeLsum': 0.132,
 'eval_runtime': 80.6454,
 'eval_samples_per_second': 1.86,
 'eval_steps_per_second': 0.93}

In [12]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,2.279200,1.972012,0.177200,0.051900,0.139300,0.147400
2,2.158700,1.921375,0.181200,0.050500,0.142100,0.148200
3,2.106400,1.904050,0.184600,0.050100,0.146800,0.155500
4,2.077700,1.895391,0.180300,0.050600,0.145000,0.152200


/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.1772, 'rouge2': 0.0519, 'rougeL': 0.1393, 'rougeLsum': 0.1474}


/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.1812, 'rouge2': 0.0505, 'rougeL': 0.1421, 'rougeLsum': 0.1482}


/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.1846, 'rouge2': 0.0501, 'rougeL': 0.1468, 'rougeLsum': 0.1555}


/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/users/eleves-b/2021/guilherme.vieira-manhaes/miniconda3/lib/python3.9/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


{'rouge1': 0.1803, 'rouge2': 0.0506, 'rougeL': 0.145, 'rougeLsum': 0.1522}


KeyboardInterrupt: 

In [ ]:
def predict(text):
    tokens = tokenizer([text], return_tensors='pt').to(device)
    output_tokens=base_model.generate(
        **tokens, 
        max_new_tokens=max_target_length, 
        temperature=2.0,
        do_sample=True,
        num_return_sequences=3
    )
    sents = [tokenizer.decode(out, skip_special_tokens=True) for out in output_tokens]
    print(sents)

def print_summary(idx):
    review = dataset["validation"][idx]["text"]
    title = dataset["validation"][idx]["titles"]
    text = base_prompt(dataset["validation"][idx]['text'])
    summary = predict(text)
    print(f"'>>> Text: {review}'")
    print(f"\n>>> Title: {title}")
    print(f"\n>>> predicted: {summary}")

In [ ]:
print_summary(100)

IndexError: Invalid key: 100 is out of bounds for size 15

In [ ]:
trainer.save_model(out_dir)

## meta model

# making prediction

In [ ]:
csv_file_path = '../data/test_text.csv'
dataset = load_dataset('csv', data_files={"data":csv_file_path})

def preprocess(examples):
    return tokenizer(
        [base_prompt(text) for text in examples["text"]],
        truncation=True,
        padding='max_length',
        max_length=max_input_length
      )

tokenized_datasets = dataset.map(preprocess, batched=True)

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [ ]:
batch_size = 8 
data_loader = DataLoader(tokenized_datasets['data'], batch_size=batch_size)

preds = []
model.eval()
with torch.no_grad():
    for bidx, batch in enumerate(data_loader):
        print(f"batch {bidx+1}/{len(data_loader)}")
        input_ids = torch.stack(batch['input_ids']).T.to(device)
        attention_mask = torch.stack(batch['attention_mask']).T.to(device)

        output_tokens = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_new_tokens=max_target_length)

        for idx, tokens in enumerate(output_tokens):
            out = tokenizer.decode(tokens, skip_special_tokens=True)
            preds.append({'ID': batch['ID'][idx].item(), "titles": out})

batch 1/188
batch 2/188
batch 3/188


KeyboardInterrupt: 

In [ ]:
test_df=pd.DataFrame(preds)


In [ ]:
import csv
filename=f"./submission-{model_name}.csv"
test_df.to_csv(filename, columns=['ID', 'titles'], index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
new_first_line = 'ID,titles\n'

# Read the contents of the CSV file
with open(filename, 'r') as file:
    lines = file.readlines()

# Substitute the first line
lines[0] = new_first_line

# Write the modified content back to the file
with open(filename, 'w') as file:
    file.writelines(lines)